#### SVD

In [14]:
!pip install Pillow tqdm

Looking in indexes: https://mirrors.tencent.com/pypi/simple/, https://mirrors.tencent.com/repository/pypi/tencent_pypi/simple
     |████████████████████████████████| 78 kB 883 kB/s             
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [1]:
import numpy as np

In [2]:
A = np.array([[0,1],[1,1],[1,0]])
A

array([[0, 1],
       [1, 1],
       [1, 0]])

In [3]:
u, s, vt = np.linalg.svd(A, full_matrices=True)
print(u.shape, s.shape, vt.shape)

(3, 3) (2,) (2, 2)


In [4]:
u

array([[-4.08248290e-01,  7.07106781e-01,  5.77350269e-01],
       [-8.16496581e-01,  7.45552182e-17, -5.77350269e-01],
       [-4.08248290e-01, -7.07106781e-01,  5.77350269e-01]])

In [5]:
s

array([1.73205081, 1.        ])

In [6]:
vt.T

array([[-0.70710678, -0.70710678],
       [-0.70710678,  0.70710678]])

In [7]:
np.allclose(A, np.dot(u[:,:2]*s, vt))

True

In [8]:
np.dot(u[:,:2]*s, vt)

array([[ 1.02120423e-16,  1.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00, -2.11898069e-16]])

In [9]:
s_ = np.zeros((3,2))
for i in range(2):
    s_[i][i] = s[i]

s_

array([[1.73205081, 0.        ],
       [0.        , 1.        ],
       [0.        , 0.        ]])

In [10]:
np.dot(np.dot(u, s_), vt)

array([[ 1.02120423e-16,  1.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00, -2.11898069e-16]])

In [23]:
!rm -rf test.jpg && wget https://img.ixintu.com/download/jpg/202002/551b30f1527ef8570025beaccdb52607.jpg -O test.jpg
!mkdir -p svd_pic

--2022-10-10 20:07:02--  https://img.ixintu.com/download/jpg/202002/551b30f1527ef8570025beaccdb52607.jpg
Resolving img.ixintu.com (img.ixintu.com)... 119.147.148.236
Connecting to img.ixintu.com (img.ixintu.com)|119.147.148.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451035 (440K) [image/jpeg]
Saving to: ‘test.jpg’

test.jpg            100%[===================>] 440.46K  --.-KB/s    in 0.03s   

2022-10-10 20:07:02 (16.2 MB/s) - ‘test.jpg’ saved [451035/451035]



In [24]:
import numpy as np
import os
from PIL import Image
from tqdm import tqdm

# 定义恢复函数，由分解后的矩阵恢复到原矩阵
def restore(u, s, v, K): 
    '''
    u:左奇异矩阵
    v:右奇异矩阵
    s:奇异值矩阵
    K:奇异值个数
    '''
    m, n = len(u), len(v[0])
    a = np.zeros((m, n))
    for k in range(K):
        uk = u[:, k].reshape(m, 1)
        vk = v[k].reshape(1, n)
        # 前k个奇异值的加总
        a += s[k] * np.dot(uk, vk)   
    a = a.clip(0, 255)
    return np.rint(a).astype('uint8')

A = np.array(Image.open("./test.jpg", 'r'))
# 对RGB图像进行奇异值分解
u_r, s_r, v_r = np.linalg.svd(A[:, :, 0])    
u_g, s_g, v_g = np.linalg.svd(A[:, :, 1])
u_b, s_b, v_b = np.linalg.svd(A[:, :, 2])

# 使用前50个奇异值
K = 50 
output_path = r'./svd_pic'
# 
for k in tqdm(range(1, K+1)):
    R = restore(u_r, s_r, v_r, k)
    G = restore(u_g, s_g, v_g, k)
    B = restore(u_b, s_b, v_b, k)
    I = np.stack((R, G, B), axis=2)   
    Image.fromarray(I).save('%s/svd_%d.jpg' % (output_path, k))

100%|██████████| 50/50 [00:10<00:00,  4.85it/s]


In [4]:
A.shape

(959, 959, 3)